# Example usage

## Version check

In [1]:
import bumbag

print(bumbag.__version__)

4.0.0


### `daycount`

Task: The digits of 22 February 2022 form [a palindrome and an ambigram](https://en.wikipedia.org/wiki/Twosday) in dd-mm-yyyy format.
List the next five dates with these properties.

In [2]:
from datetime import date

from toolz import curried

import bumbag


def format_date(date, with_dash):
    fmt = "%d-%m-%Y" if with_dash else "%d%m%Y"
    return date.strftime(fmt)


def is_palindrome_date(date):
    d = format_date(date, False)
    return d == d[::-1]


def is_ambigram_date(date):
    d = format_date(date, False)
    return set(d) <= {"0", "1", "2", "8"}


def show_date(date):
    return f"{format_date(date, True)} ↦ {format_date(date, False)}"


curried.pipe(
    bumbag.daycount(date(2022, 2, 23)),
    curried.filter(is_palindrome_date),
    curried.filter(is_ambigram_date),
    curried.map(show_date),
    curried.take(5),
    list,
)

['08-02-2080 ↦ 08022080',
 '18-02-2081 ↦ 18022081',
 '28-02-2082 ↦ 28022082',
 '10-12-2101 ↦ 10122101',
 '20-12-2102 ↦ 20122102']

### `flatten`

In [3]:
import bumbag

irregular_list = [
    ["one", 2],
    3,
    [(4, "five")],
    [[["six"]]],
    "seven",
    [],
]

list(bumbag.flatten(irregular_list, 8, [9, ("ten",)]))

['one', 2, 3, 4, 'five', 'six', 'seven', 8, 9, 'ten']

### `highlight_string_differences`

Use in an `assert` statement to get more information:

In [4]:
import traceback

import bumbag

lft_str = "hello"
rgt_str = "hallo"

Common approach to print values might not be informative enough for subtle differences.

In [5]:
try:
    assert lft_str == rgt_str, f"{lft_str} != {rgt_str}"
except AssertionError:
    traceback.print_exc()

Traceback (most recent call last):
  File "/tmp/ipykernel_22739/54865336.py", line 2, in <cell line: 1>
    assert lft_str == rgt_str, f"{lft_str} != {rgt_str}"
AssertionError: hello != hallo


With `highlight_string_differences`, it is easier to spot subtle differences.

In [6]:
def get_string_diff(lft_str, rgt_str):
    return "lft_str != rgt_str\n" + bumbag.highlight_string_differences(
        lft_str, rgt_str
    )


try:
    assert lft_str == rgt_str, get_string_diff(lft_str, rgt_str)
except AssertionError:
    traceback.print_exc()

Traceback (most recent call last):
  File "/tmp/ipykernel_22739/4023425549.py", line 8, in <cell line: 7>
    assert lft_str == rgt_str, get_string_diff(lft_str, rgt_str)
AssertionError: lft_str != rgt_str
hello
 |   
hallo


### `stopwatch`

In [7]:
import functools
from datetime import datetime

import time_machine

# overwrite defaults for illustrative purposes
stopwatch = functools.partial(
    bumbag.stopwatch,
    fmt="%a, %-d %b %Y %H:%M:%S",
    flush=False,
)

# mock datetime
traveller = time_machine.travel(datetime(2023, 1, 1, 12, 0, 0))
traveller.start();

Use as context manager.

Example 1: Measure total elapsed time of multiple statements.

In [8]:
import time

with stopwatch("example 1"):
    time.sleep(0.05)
    time.sleep(0.05)

Sun, 1 Jan 2023 12:00:00 -> Sun, 1 Jan 2023 12:00:00 = 0.100421s - example 1


Example 2: Measure total elapsed time of multiple `stopwatch` instances.

In [9]:
with stopwatch("example 2 - stopwatch 1") as sw1:
    time.sleep(0.05)

with stopwatch("example 2 - stopwatch 2") as sw2:
    time.sleep(0.05)

sw1 + sw2

Sun, 1 Jan 2023 12:00:00 -> Sun, 1 Jan 2023 12:00:00 = 0.050203s - example 2 - stopwatch 1
Sun, 1 Jan 2023 12:00:00 -> Sun, 1 Jan 2023 12:00:00 = 0.050605s - example 2 - stopwatch 2


0.100808s - total elapsed time

Use as decorator.

In [10]:
@stopwatch("example 3")
def func_with_supplied_label():
    time.sleep(0.1)

In [11]:
func_with_supplied_label()

Sun, 1 Jan 2023 12:00:00 -> Sun, 1 Jan 2023 12:00:00 = 0.100273s - example 3


In [12]:
@stopwatch()
def func_with_no_supplied_label():
    time.sleep(0.1)

In [13]:
func_with_no_supplied_label()

Sun, 1 Jan 2023 12:00:00 -> Sun, 1 Jan 2023 12:00:00 = 0.100257s - func_with_no_supplied_label


In [14]:
traveller.stop()

### `two_set_summary`

In [15]:
import bumbag

x = {"a", "c", "b", "g", "h"}
y = {"c", "d", "e", "f", "g"}
summary = bumbag.two_set_summary(x, y)
print(summary["report"])

    x (n=5): {'a', 'b', 'c', ...}
    y (n=5): {'c', 'd', 'e', ...}
x | y (n=8): {'a', 'b', 'c', ...}
x & y (n=2): {'c', 'g'}
x - y (n=3): {'a', 'b', 'h'}
y - x (n=3): {'d', 'e', 'f'}
x ^ y (n=6): {'a', 'b', 'd', ...}
jaccard = 0.25
overlap = 0.4
dice = 0.4
disjoint?: False
x == y: False
x <= y: False
x <  y: False
y <= x: False
y <  x: False
